In [ ]:
!pip install antigranular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00


In [ ]:
import antigranular as ag
session = ag.login("***", "***", competition = "Harvard OpenDP Hackathon")

Dataset "Flight Company Dataset" loaded to the kernel as flight_company_dataset

Dataset "Health Organisation Dataset" loaded to the kernel as health_organisation_dataset

Connected to Antigranular server session id: 746fd2d0-8847-4d4a-9adc-950c88b78110, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
%%ag
health = health_organisation_dataset
flight = flight_company_dataset

In [ ]:
%%ag
# Lets remove those passenger records who tested negative.
health['covidtest_result'] = health['covidtest_result'].where(health['covidtest_result'] == 'positive')
health = health.dropna()

In this notebook, I'll share the methods I've tried. For the ones who wants a short explanation, the key takeaway: dates played a crucial role in the process.

Let's begin with how I handled dates. The essential step was to accurately extract the day, month, and year for the linking. Extracting the day and month was straightforward; however, extracting the year presented challenges. For instance, differentiating between '1910-2010' and '10' was not simple. My solution was to consistently extract the last two digits of any year, treating them as '10' in all cases. This approach significantly improved my linking accuracy.

I also created a full date feature by combining them all but for some reason it didn't work and I was too lazy to fix it.

In [ ]:
%%ag

import re
import pandas as pd

def find_day_in_date(input_date: str) -> str:
    day_match = re.search(r'\b(0?[1-9]|[12][0-9]|3[01])\b', input_date)
    return day_match.group(0).zfill(2) if day_match else input_date

def find_month_in_date(input_date: str) -> str:
    # Dictionary mapping month names and abbreviations to their numerical equivalents
    months_dict = {
        "jan": "01", "feb": "02", "mar": "03", "apr": "04",
        "may": "05", "jun": "06", "jul": "07", "aug": "08",
        "sep": "09", "oct": "10", "nov": "11", "dec": "12"
    }
    months_dict.update({month.capitalize(): num for month, num in months_dict.items()})
    months_dict.update({month.upper(): num for month, num in months_dict.items()})

    # Regular expression pattern to match month names or abbreviations
    month_pattern = '|'.join(months_dict.keys())
    match = re.search(month_pattern, input_date)

    if match:
        # Get the matched month text and find its numerical representation
        month_text = match.group(0)
        return months_dict.get(month_text)

    # If no month name is found, try to extract numeric month format
    numeric_month_match = re.search(r'\b(0[1-9]|1[0-2])\b', input_date)
    if numeric_month_match:
        return numeric_month_match.group()

    return input_date

def find_year_in_date(input_date: str) -> str:
    # Search for the year component at the end of the string
    year_match = re.search(r'([0-9]{2,4})$', input_date)
    if year_match:
        year = year_match.group(0)
        # Return the last two digits of the year
        return year[-2:]
    return input_date

def find_full_date(input_date: str) -> str:
    day = find_day_in_date(input_date)
    month = find_month_in_date(input_date)
    year = find_year_in_date(input_date)
    if day and month and year:
        return f"{year}-{month}-{day}"
    return input_date

In [ ]:
%%ag
flight['passenger_day'] = flight['passenger_date_of_birth'].map(find_day_in_date)
health['patient_day'] = health['patient_date_of_birth'].map(find_day_in_date)

flight['passenger_month'] = flight['passenger_date_of_birth'].map(find_month_in_date)
health['patient_month'] = health['patient_date_of_birth'].map(find_month_in_date)

flight['passenger_year'] = flight['passenger_date_of_birth'].map(find_year_in_date)
health['patient_year'] = health['patient_date_of_birth'].map(find_year_in_date)

flight['passenger_full_date'] = flight['passenger_date_of_birth'].map(find_full_date)
health['patient_full_date'] = health['patient_date_of_birth'].map(find_full_date)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In names, when a second uppercase letter comes up, I drop all the remaining elements and convert everything to lowercase.
This allows Nok, NokVanu and Nok Vanu to be returned as nok.
This didn't improve the score at all and that was surprising to me tbh.

In [ ]:
%%ag
import re
import pandas as pd

def reduce_to_first_name(name: str) -> str:
    # Using regex to extract the first name before a space or a second uppercase letter
    first_name_match = re.match(r"([A-Za-z]+)(?=\s|[A-Z])", name)
    if first_name_match:
        return first_name_match.group(0).lower()
    else:
        # If no space or second uppercase letter is found, return the name as is but lowercase
        return name.lower()

flight['passenger_firstname'] = flight['passenger_firstname'].map(reduce_to_first_name)
health['patient_firstname'] = health['patient_firstname'].map(reduce_to_first_name)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



Same goes for surnames as well. Which makes sense. Names had way more potential.

In [ ]:
# %%ag
# def format_surname(surname: str) -> str:
#     # If the first letter is uppercase, remove everything after the second uppercase letter
#     if surname[0].isupper():
#         formatted_surname = re.match(r"[A-Z][a-z]+", surname)
#     else:
#         # If the first letter is not uppercase, remove everything after the first uppercase letter
#         formatted_surname = re.match(r"[a-z]+", surname)

#     return formatted_surname.group(0).lower() if formatted_surname else surname.lower()

# flight['passenger_lastname'] = flight['passenger_lastname'].map(format_surname)
# health['patient_lastname'] = health['patient_lastname'].map(format_surname)

In emails, I remove the elements after @ and turn all characters that are numbers or not numbers into _. Then I make them all lowercase letters.

This didn't work either. Probably caused oversimplification and didn't create proper links

In [ ]:
%%ag

def normalize_email(email: str) -> str:
    # Remove everything after '@'
    email_without_domain = email.split('@')[0]

    # Replace non-alphanumeric characters with '_'
    normalized_email = re.sub(r'[^a-zA-Z0-9]', '_', email_without_domain)

    # Convert to lowercase
    return normalized_email.lower()

flight['normalized_passenger_email'] = flight['passenger_email_address'].map(normalize_email)
health['normalized_patient_email'] = health['patient_email_address'].map(normalize_email)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



For phones, I removed all non-number values ​​and then changed the observations that started with +, but while I was preparing the notebook, I realized that the second part will not work because I removed the + character in the first part.

This format didn't work anyway. The reason is clear lol.

In [ ]:
%%ag
def format_phone_number(phone_number: str) -> str:
    # Remove non-numeric characters except leading '+'
    cleaned_number = re.sub(r'(?<!^\+)[^0-9]', '', phone_number)

    # If number starts with '+', replace it with '00'
    if cleaned_number.startswith('+'):
        cleaned_number = '00' + cleaned_number[1:]

    return cleaned_number

# Apply the function to the DataFramepatient_phone_number
flight['passenger_phone_number'] = flight['passenger_phone_number'].map(format_phone_number)
health['patient_phone_number'] = health['patient_phone_number'].map(format_phone_number)

/code/dependencies/op_pandas/op_pandas/core/private_dataframe.py:512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[key] = value._series



In [ ]:
%%ag
# Printing the column names using ag_print
ag_print(f"Flight Dataset \n {flight.columns} \n")
ag_print(f"Health Dataset \n {health.columns}")

Flight Dataset 
 ['flight_number', 'flight_date', 'flight_from', 'flight_to', 'passenger_firstname', 'passenger_lastname', 'passenger_date_of_birth', 'passenger_phone_number', 'passenger_email_address', 'passenger_day', 'passenger_month', 'passenger_year', 'passenger_full_date', 'normalized_passenger_email'] 

Health Dataset 
 ['patient_firstname', 'patient_lastname', 'patient_date_of_birth', 'patient_phone_number', 'patient_email_address', 'covidtest_date', 'covidtest_result', 'patient_address', 'patient_day', 'patient_month', 'patient_year', 'patient_full_date', 'normalized_patient_email']



There's not much to tell here. I tried many different blocks. These were the ones that worked best.

In [ ]:
%%ag
import op_recordlinkage as rl
indexer = rl.Index()

indexer.block(['passenger_day','passenger_month', 'passenger_year'],['patient_day','patient_month', 'patient_year'])
indexer.block(['passenger_firstname','passenger_lastname'],['patient_firstname','patient_lastname'])
# indexer.block('patient_date_full','passenger_date_full')
# indexer.block(['passenger_day','passenger_month'],['patient_day','patient_month'])
# indexer.block('passenger_phone_number','patient_phone_number')
# indexer.block('passenger_email_address','patient_email_address')
# indexer.block('normalized_passenger_email','normalized_patient_email')
# indexer.block('passenger_firstname2','patient_firstname2')

candidate_links = indexer.index(flight,health)

Likewise here, day, month and year should definitely be included in the compares. Names work well with date, but phone and email might be better instead. Since I was satisfied with my score, I did not try to improve it further.

Also cmp increases the score slightly. Not including the 14th day also increased the score slightly.

In [ ]:
%%ag
compare = rl.Compare()

# Using inbuilt string linking via fuzzy match. ( keeping threshold for last_name to get stronger links )
compare.string("passenger_firstname" , "patient_firstname", method='jarowinkler', threshold=0.7, label="firstname")
compare.string("passenger_lastname","patient_lastname", method='jarowinkler', threshold=0.7, label="lastname")
# compare.string("passenger_email_address","patient_email_address" ,method='jarowinkler', threshold=0.6, label="mailaddress")
# compare.string("passenger_phone_number","patient_phone_number" ,method='jarowinkler', threshold=0.4, label="phone")
compare.exact("passenger_day","patient_day", label="day")
compare.exact("passenger_month","patient_month", label="month")
compare.exact("passenger_year","patient_year", label="year")
# compare.string("passenger_year","patient_year",method='jarowinkler', threshold=0.5, label="year")
# compare.string("patient_date_full","passenger_date_full" ,method='jarowinkler', threshold=0.75, label="year")
# compare.string("passenger_date_of_birth","patient_date_of_birth" ,method='jarowinkler', threshold=0.6, label="dob")

# Using a custom compare rule.
# custom functions are executed in isolated environment.
import datetime
def cmp(date_str1:str , date_str2:str)->int: # datetime and regex are pre-imported in isolated environment.
    # Convert date strings to datetime objects
    date1 = datetime.datetime.strptime(date_str1, "%Y-%m-%d")
    date2 = datetime.datetime.strptime(date_str2, "%Y-%m-%d")

    # Calculate the absolute difference in days
    days_apart = (date2 - date1).days
    # Check if the dates are within two weeks (13 days) apart
    if -13 <= days_apart <= 13:
        return 1
    else:
        return 0

compare.custom(cmp,"flight_date","covidtest_date",label="date_cmp")

In [ ]:
%%ag
features = compare.compute(candidate_links,flight,health)

It's worth talking about get_match(). If you look at the compare matches above, you will see that I wrote 6 different conditions.
I want all of these to be matched for the link to be established. That's why I enter a value above 5 in get_match

In [ ]:
%%ag
linked_df = compare.get_match(5.5)

In [ ]:
%%ag
# Submitting the column containing the filtered set of airlines we should report regarding a covid passenger.
res = linked_df[["l_flight_number"]]
x = submit_predictions(res)

score: {'leaderboard': 0.9496866036671211, 'logs': {'LIN_EPS': -0.0055000000000000005, 'MCC': 0.955186603667121}}



In [ ]:
# session.terminate_session()

{'status': 'ok'}